In [1]:
import fastText
import math
import linecache
import numpy as np 
from numpy import random
from random import sample
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import *
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

# from attention_utils import get_activations, get_data_recurrent


/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [3]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()
nb_sequence_length = 75

In [62]:
def twitter_tokenizer(textline):
    textLine = re.sub(r'http\S+', 'URL', textline)
    textline = re.sub('@[\w_]+', 'USER_MENTION', textline)
    textline = re.sub('\|LBR\|', '', textline)
    textline = re.sub('\.\.\.+', '...', textline)
    textline = re.sub('!!+', '!!', textline)
    textline = re.sub('\?\?+', '??', textline)
    words = re.compile('[\U00010000-\U0010ffff]|[\w-]+|[^ \w\U00010000-\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    # print(words)
    return(words)

In [63]:
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims, tokenize=True):
    if not tokenize:
        words = textline.split()
    else:
        words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_idx = np.zeros(nb_sequence_length)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    return features_ft

In [68]:
def sequential_generator(filename, 
                         batch_size, 
                         labels2Idx:'dict to make output labels',
                         check:'to check if all lines in file are of same length.To check enter the len of line after splitting it by tabs' = None,
                         tokenize:'specify if using twitter tokenzor to preprocess lines'=False, 
                        ):    
    
    f = open(filename)
    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
#         print(type(labels2Idx))
        batch_labels = np.zeros((batch_size, len(labels2Idx)))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
            data = line.strip().split('\t')
            if check:
                if len(data)!=check:
                    i-=1
                    continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims, tokenize= tokenize)
            if len(labels2Idx)==2:
                batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            else:
                batch_labels[i] = to_categorical(labels2Idx[data[1]], n_labels)
        yield ([batch_features_ft], batch_labels)

In [69]:
def train_dev_sentences(filetrain,filedev, check:'to check if lines of file are all same lenght after separating by tab'):
    labels2Idx = {}
    train_lines = [line.strip().split("\t") for line in open(filetrain) if len(line.strip().split('\t'))==check]
    dev_lines = [line.strip().split("\t") for line in open(filedev) if len(line.strip().split('\t'))==check]

    train_sentences = [x[0] for x in train_lines]
    for dataset in [train_lines, dev_lines]:
        for line in dataset:
            label = line[1]
            if label not in labels2Idx.keys():
                labels2Idx[label]= len(labels2Idx)
                
#     train_labels = [0 if x[1] == "OTHER" else 1 for x in train_lines]
    train_labels = [labels2Idx[x[1]] for x in train_lines]
    dev_sentences = [x[0] for x in dev_lines]
#     dev_labels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]
    dev_labels = [labels2Idx[x[1]] for x in dev_lines]
    return (train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx)


In [43]:
file_train = '/home/jindal/notebooks/twitter_data/twitter_classes_500k_train.csv'
file_dev = '/home/jindal/notebooks/twitter_data/twitter_classes_500k_dev.csv'
train_sentences, train_labels, dev_sentences, dev_labels, labels2Id = train_dev_sentences(filetrain=file_train,filedev=file_dev, check=2)

In [44]:
print(len(dev_sentences))

100000


In [45]:
dev_labels[:10]

[421, 10, 274, 329, 134, 462, 369, 91, 273, 362]

In [46]:
print(len(dev_labels))

100000


In [47]:
n_labels = 500

In [110]:
def compile_model(no_labels:'total labels for classification'):
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)

    filter_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in filter_sizes:
        conv = Conv1D(
            filters = 200,
            kernel_size = sz,
            padding = 'valid',
            strides = 1
        )(lstm_block)
        conv = LeakyReLU()(conv)
        conv = GlobalMaxPooling1D()(conv)
        conv = Dropout(0.5)(conv)
        conv_blocks.append(conv)
    model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    # model_concatenated = Dropout(0.8)(model_concatenated)
    model_concatenated = Dense(100)(model_concatenated)
    model_concatenated = LeakyReLU()(model_concatenated)
    model_output = Dense(no_labels, activation = "softmax")(model_concatenated)
    new_model = Model(model_input_embedding, model_output)
    new_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
#     new_model.summary()
    return new_model

In [71]:
print(dev_sentences[1])

RT USER_MENTION : URL / i2a32rRGUt


In [85]:
def test_model(generator, 
               train_sentences, 
               devLabels, 
               number_of_tests,
               number_of_epochs,
               filename_to_log, 
               labels_earlier:'number of original labels if loading a pretrained model'
               filename_to_save_weigths,
               batch_size, 
               train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', 
               pos_label:'only if binary f1',
               labels2Idx,
               load_model_weights=False,
               model_weights_file:'give filepath as str'=None, 
               tokenize=True,
               nb_sequence_length = nb_sequence_length,
               nb_embedding_dims= nb_embedding_dims, 
               check_for_generator=None,
                ):
    
#     f = open(filename_to_log,"w")
    
    max_f1=0
    max_p=0
    max_r=0
    max_a=0
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model(labels_earlier)

        # transfer learning
        if load_model_weights and model_weights_file:
                model.load_weights(model_weights_file)

        samples_per_epoch = len(train_sentences)
        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
        checkpoint = ModelCheckpoint(filename_to_save_weigths, monitor='val_acc',save_best_only = True, 
                                     save_weights_only = True)

        for epoch in range(epochs):
            print("Epoch: %d" %(epoch+1))
            model.fit_generator(
                generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                          labels2Idx= labels2Idx,tokenize= tokenize), 
                steps_per_epoch= steps_per_epoch, epochs=1,
                validation_data = generator(filename ='/home/jindal/notebooks/twitter_data/twitter_classes_500k_dev.csv', 
                                            batch_size = batch_size, check = check_for_generator, 
                                           labels2Idx = labels2Idx, tokenize = tokenize),
                validation_steps = math.ceil(len(dev_labels) / batch_size),
                callbacks = [checkpoint]
            )

#             testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
#             for i in range(len(dev_sentences)):
#                 testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
#             results = model.predict(testset_features)


# #             idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
#             predLabels = results.argmax(axis=-1)
#             devLabels = devLabels
#             f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
#             r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
#             p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
#             a = accuracy_score(devLabels, predLabels)
#             if max_f1 < f1:
#                 print("model saved. F1 is %f" %(f1))
#                 model.save(filename_to_save_weigths)
#                 max_f1 = f1
#                 max_p = p
#                 max_r = r
#                 max_a = a
#             text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
#             print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
#         to_write= "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
#         print(to_write)
#         f.write(to_write)
#         total_f1+=max_f1
#         total_prec+=max_p
#         total_acc+=max_a
#         total_recall+=max_r    
#         print("*****************************************************************************")
#     final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
#     print(final_text)
#     f.write(final_text)
#     f.close()




In [86]:
generator = sequential_generator
train_sentences = train_sentences
devLabels = dev_labels
number_of_tests = 1
number_of_epochs = 10
twitterclasses_pretraining_log = '/home/jindal/notebooks/jindal/NER/language_model/results_pretraining_twitterclasses.txt' 
twitterclasses_pretraining_save_weigths='/home/jindal/notebooks/jindal/NER/language_model/model_pretrained_twitterclasses.h5'
batch_size=32*4
twitterclasses_train_file='/home/jindal/notebooks/twitter_data/twitter_classes_500k_train.csv'
tokenize = False
labels2Idx = labels2Id
f1_measure='macro'
pos_label=0
load_model_weights=False
# model_weights_file:'give filepath as str'=None, 
nb_sequence_length = nb_sequence_length
nb_embedding_dims= nb_embedding_dims
check_for_generator=2



In [87]:
len(labels2Idx)

500

In [88]:
test_model(generator=generator, 
           train_sentences=train_sentences, 
           devLabels=devLabels,
           number_of_tests= number_of_tests,
          number_of_epochs=number_of_epochs,
           filename_to_log=twitterclasses_pretraining_log, 
           filename_to_save_weigths=twitterclasses_pretraining_save_weigths,
           tokenize= tokenize, 
           labels2Idx= labels2Idx,
          batch_size=batch_size,
           train_file=twitterclasses_train_file, 
           f1_measure=f1_measure, 
           pos_label=pos_label, 
           load_model_weights=load_model_weights,
          nb_sequence_length=nb_sequence_length,
           nb_embedding_dims=nb_embedding_dims, 
           check_for_generator= check_for_generator)

Test 1/1
Epoch: 1
Epoch 1/1
3125/3125 [==============================] - 780s 250ms/step - loss: 4.1036 - acc: 0.2424 - val_loss: 3.8253 - val_acc: 0.2802
Epoch: 2
Epoch 1/1
3125/3125 [==============================] - 810s 259ms/step - loss: 3.7542 - acc: 0.2989 - val_loss: 3.6883 - val_acc: 0.3025
Epoch: 3
Epoch 1/1
3125/3125 [==============================] - 815s 261ms/step - loss: 3.6423 - acc: 0.3167 - val_loss: 3.6047 - val_acc: 0.3182
Epoch: 4
Epoch 1/1
3125/3125 [==============================] - 811s 260ms/step - loss: 3.5797 - acc: 0.3264 - val_loss: 3.5725 - val_acc: 0.3198
Epoch: 5
Epoch 1/1
3125/3125 [==============================] - 821s 263ms/step - loss: 3.5343 - acc: 0.3336 - val_loss: 3.5454 - val_acc: 0.3239
Epoch: 6
Epoch 1/1
3125/3125 [==============================] - 793s 254ms/step - loss: 3.4973 - acc: 0.3394 - val_loss: 3.5259 - val_acc: 0.3284
Epoch: 7
Epoch 1/1
3125/3125 [==============================] - 785s 251ms/step - loss: 3.4718 - acc: 0.3430 - val_

# TRANSFER LEARNING HERE


In [126]:
def test_model_tl(generator, 
               train_sentences, 
               devLabels, 
               number_of_tests,
               number_of_epochs,
               filename_to_log, 
               filename_to_save_weigths,
               batch_size, 
               train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', 
               pos_label:'only if binary f1',
               load_model_weights=False,
               model_weights_file:'give filepath as str'=None, 
               tokenize=True,
               nb_sequence_length = nb_sequence_length,
               nb_embedding_dims= nb_embedding_dims, 
               check_for_generator=None,
                ):
    
    f = open(filename_to_log,"w")
    
   
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model(500)

        # transfer learning
        if load_model_weights and model_weights_file:
                model.load_weights(model_weights_file)
                print("removing top layer")
                model.layers.pop()
                output = Dense(2, activation = 'softmax')(model.layers[-1].output)
                final_model = Model(inputs=model.input, outputs=[output])
                final_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
#                 for layer in final_model.layers:
#                     print(layer.name)

        samples_per_epoch = len(train_sentences)
        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
#         checkpoint = ModelCheckpoint(filename_to_save_weigths, monitor='val_acc',save_best_only = True, 
#                                      save_weights_only = True)


        max_f1=0
        max_p=0
        max_r=0
        max_a=0
        for epoch in range(epochs):
            print("Epoch: %d" %(epoch+1))
            final_model.fit_generator(
                generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                          labels2Idx= labels2Idx,tokenize= tokenize), 
                steps_per_epoch= steps_per_epoch, epochs=1,
#                 validation_data = generator(filename ='/home/jindal/notebooks/twitter_data/twitter_classes_500k_dev.csv', 
#                                             batch_size = batch_size, check = check_for_generator, 
#                                            labels2Idx = labels2Idx, tokenize = tokenize),
#                 validation_steps = math.ceil(len(dev_labels) / batch_size),
#                 callbacks = [checkpoint]
            )

            testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
            for i in range(len(dev_sentences)):
                testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
            results = final_model.predict(testset_features)


#             idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
            predLabels = results.argmax(axis=-1)
            devLabels = devLabels
            f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
            r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
            p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
            a = accuracy_score(devLabels, predLabels)
            if max_f1 < f1:
                print("model saved. F1 is %f" %(f1))
                final_model.save(filename_to_save_weigths)
                max_f1 = f1
                max_p = p
                max_r = r
                max_a = a
            text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
            print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
        to_write= "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
        print(to_write)
        f.write(to_write)
        total_f1+=max_f1
        total_prec+=max_p
        total_acc+=max_a
        total_recall+=max_r    
        print("*****************************************************************************")
    final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
    print(final_text)
    f.write(final_text)
    f.close()




In [127]:
n_labels =2

In [128]:
train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx = train_dev_sentences(filetrain='/home/gwiedemann/notebooks/OffLang/sample_train.txt',
                   filedev='/home/gwiedemann/notebooks/OffLang/sample_dev.txt', check=3)

In [129]:
print(dev_sentences[0])
print(dev_labels[:20])

@FilmElf_yt Diese ganzen leute die du angesprochen hast sind untermenschen ja aber das alte Deutschland und deutsche volk waren ehrenwerte leute und ein stolzes Land
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1]


In [130]:
generator = sequential_generator
train_sentences = train_sentences
devLabels = dev_labels
number_of_tests = 5
number_of_epochs = 50
twitterclasses_tl_log = '/home/jindal/notebooks/jindal/NER/language_model/results_tl_twitterclasses.txt' 
twitterclasses_tl_save_weigths='/home/jindal/notebooks/jindal/NER/language_model/classification_model_tl_twitterclasses.h5'
batch_size=32
twitterclasses_tl_train_file='/home/gwiedemann/notebooks/OffLang/sample_train.txt'
f1_measure='binary'
pos_label=1
load_model_weights=True
model_weights_file = '/home/jindal/notebooks/jindal/NER/language_model/model_pretrained_twitterclasses.h5'
nb_sequence_length = nb_sequence_length
nb_embedding_dims= nb_embedding_dims
check_for_generator=3

In [131]:
test_model_tl(generator=generator, 
           train_sentences=train_sentences, 
           devLabels=devLabels, 
           number_of_tests= number_of_tests,
           number_of_epochs=number_of_epochs, 
           filename_to_log=twitterclasses_tl_log, 
           filename_to_save_weigths=twitterclasses_tl_save_weigths,
           batch_size=batch_size,
           train_file=twitterclasses_tl_train_file, 
           f1_measure=f1_measure, 
           pos_label=pos_label, 
           load_model_weights=load_model_weights,
           model_weights_file = model_weights_file, 
           nb_sequence_length=nb_sequence_length, 
           nb_embedding_dims=nb_embedding_dims, 
           check_for_generator= check_for_generator)

Test 1/5
removing top layer
Epoch: 1
Epoch 1/1
132/132 [==============================] - 34s 258ms/step - loss: 0.6235 - acc: 0.6664
model saved. F1 is 0.527273
Test-Data: Prec: 0.671, Rec: 0.434, F1: 0.527, Acc: 0.743
Epoch: 2
Epoch 1/1
132/132 [==============================] - 31s 231ms/step - loss: 0.5196 - acc: 0.7431
model saved. F1 is 0.625954
Test-Data: Prec: 0.638, Rec: 0.614, F1: 0.626, Acc: 0.757
Epoch: 3
Epoch 1/1
132/132 [==============================] - 31s 234ms/step - loss: 0.4744 - acc: 0.7663
model saved. F1 is 0.650980
Test-Data: Prec: 0.683, Rec: 0.622, F1: 0.651, Acc: 0.780
Epoch: 4
Epoch 1/1
132/132 [==============================] - 30s 231ms/step - loss: 0.4515 - acc: 0.7808
model saved. F1 is 0.677043
Test-Data: Prec: 0.704, Rec: 0.652, F1: 0.677, Acc: 0.795
Epoch: 5
Epoch 1/1
132/132 [==============================] - 30s 227ms/step - loss: 0.4209 - acc: 0.8054
model saved. F1 is 0.694656
Test-Data: Prec: 0.708, Rec: 0.682, F1: 0.695, Acc: 0.802
Epoch: 6
Epo

132/132 [==============================] - 31s 232ms/step - loss: 0.0523 - acc: 0.9818
Test-Data: Prec: 0.654, Rec: 0.659, F1: 0.657, Acc: 0.772
Epoch: 50
Epoch 1/1
132/132 [==============================] - 33s 247ms/step - loss: 0.0451 - acc: 0.9860
Test-Data: Prec: 0.665, Rec: 0.655, F1: 0.660, Acc: 0.777
prec: 0.7213740458015268 rec: 0.7078651685393258 f1: 0.7145557655954632 acc: 0.8131188118811881 

*****************************************************************************
Test 2/5
removing top layer
Epoch: 1
Epoch 1/1
132/132 [==============================] - 37s 281ms/step - loss: 0.6204 - acc: 0.6714
model saved. F1 is 0.533917
Test-Data: Prec: 0.642, Rec: 0.457, F1: 0.534, Acc: 0.736
Epoch: 2
Epoch 1/1
132/132 [==============================] - 33s 250ms/step - loss: 0.5328 - acc: 0.7268
model saved. F1 is 0.609808
Test-Data: Prec: 0.708, Rec: 0.536, F1: 0.610, Acc: 0.774
Epoch: 3
Epoch 1/1
132/132 [==============================] - 33s 252ms/step - loss: 0.4774 - acc: 0.7

132/132 [==============================] - 30s 228ms/step - loss: 0.0473 - acc: 0.9832
Test-Data: Prec: 0.685, Rec: 0.708, F1: 0.696, Acc: 0.796
Epoch: 47
Epoch 1/1
132/132 [==============================] - 30s 229ms/step - loss: 0.0529 - acc: 0.9808
model saved. F1 is 0.698182
Test-Data: Prec: 0.678, Rec: 0.719, F1: 0.698, Acc: 0.795
Epoch: 48
Epoch 1/1
132/132 [==============================] - 30s 225ms/step - loss: 0.0464 - acc: 0.9825
Test-Data: Prec: 0.698, Rec: 0.648, F1: 0.672, Acc: 0.791
Epoch: 49
Epoch 1/1
132/132 [==============================] - 30s 228ms/step - loss: 0.0527 - acc: 0.9806
Test-Data: Prec: 0.675, Rec: 0.678, F1: 0.677, Acc: 0.786
Epoch: 50
Epoch 1/1
132/132 [==============================] - 30s 228ms/step - loss: 0.0481 - acc: 0.9827
Test-Data: Prec: 0.669, Rec: 0.704, F1: 0.686, Acc: 0.787
prec: 0.6784452296819788 rec: 0.7191011235955056 f1: 0.6981818181818182 acc: 0.7945544554455446 

*********************************************************************

132/132 [==============================] - 30s 224ms/step - loss: 0.0579 - acc: 0.9789
Test-Data: Prec: 0.693, Rec: 0.659, F1: 0.676, Acc: 0.791
Epoch: 44
Epoch 1/1
132/132 [==============================] - 30s 224ms/step - loss: 0.0538 - acc: 0.9796
Test-Data: Prec: 0.718, Rec: 0.659, F1: 0.687, Acc: 0.802
Epoch: 45
Epoch 1/1
132/132 [==============================] - 29s 223ms/step - loss: 0.0539 - acc: 0.9806
Test-Data: Prec: 0.705, Rec: 0.644, F1: 0.673, Acc: 0.793
Epoch: 46
Epoch 1/1
132/132 [==============================] - 30s 226ms/step - loss: 0.0535 - acc: 0.9811
Test-Data: Prec: 0.726, Rec: 0.614, F1: 0.665, Acc: 0.796
Epoch: 47
Epoch 1/1
132/132 [==============================] - 29s 223ms/step - loss: 0.0593 - acc: 0.9782
Test-Data: Prec: 0.690, Rec: 0.693, F1: 0.692, Acc: 0.796
Epoch: 48
Epoch 1/1
132/132 [==============================] - 30s 225ms/step - loss: 0.0472 - acc: 0.9825
Test-Data: Prec: 0.702, Rec: 0.678, F1: 0.690, Acc: 0.798
Epoch: 49
Epoch 1/1
132/132 [=

132/132 [==============================] - 30s 226ms/step - loss: 0.0647 - acc: 0.9761
Test-Data: Prec: 0.640, Rec: 0.667, F1: 0.653, Acc: 0.766
Epoch: 41
Epoch 1/1
132/132 [==============================] - 30s 226ms/step - loss: 0.0550 - acc: 0.9815
Test-Data: Prec: 0.665, Rec: 0.648, F1: 0.657, Acc: 0.776
Epoch: 42
Epoch 1/1
132/132 [==============================] - 29s 223ms/step - loss: 0.0591 - acc: 0.9766
Test-Data: Prec: 0.687, Rec: 0.667, F1: 0.677, Acc: 0.790
Epoch: 43
Epoch 1/1
132/132 [==============================] - 30s 224ms/step - loss: 0.0616 - acc: 0.9761
Test-Data: Prec: 0.667, Rec: 0.674, F1: 0.670, Acc: 0.781
Epoch: 44
Epoch 1/1
132/132 [==============================] - 30s 226ms/step - loss: 0.0535 - acc: 0.9789
Test-Data: Prec: 0.731, Rec: 0.610, F1: 0.665, Acc: 0.797
Epoch: 45
Epoch 1/1
132/132 [==============================] - 29s 223ms/step - loss: 0.0537 - acc: 0.9796
Test-Data: Prec: 0.629, Rec: 0.742, F1: 0.680, Acc: 0.770
Epoch: 46
Epoch 1/1
132/132 [=

132/132 [==============================] - 30s 226ms/step - loss: 0.0739 - acc: 0.9718
Test-Data: Prec: 0.690, Rec: 0.640, F1: 0.664, Acc: 0.786
Epoch: 39
Epoch 1/1
132/132 [==============================] - 30s 226ms/step - loss: 0.0736 - acc: 0.9740
Test-Data: Prec: 0.678, Rec: 0.700, F1: 0.689, Acc: 0.791
Epoch: 40
Epoch 1/1
132/132 [==============================] - 30s 226ms/step - loss: 0.0606 - acc: 0.9759
Test-Data: Prec: 0.723, Rec: 0.644, F1: 0.681, Acc: 0.801
Epoch: 41
Epoch 1/1
132/132 [==============================] - 30s 227ms/step - loss: 0.0596 - acc: 0.9754
Test-Data: Prec: 0.652, Rec: 0.682, F1: 0.667, Acc: 0.775
Epoch: 42
Epoch 1/1
132/132 [==============================] - 30s 226ms/step - loss: 0.0565 - acc: 0.9775
Test-Data: Prec: 0.662, Rec: 0.674, F1: 0.668, Acc: 0.778
Epoch: 43
Epoch 1/1
132/132 [==============================] - 30s 226ms/step - loss: 0.0601 - acc: 0.9773
Test-Data: Prec: 0.685, Rec: 0.693, F1: 0.689, Acc: 0.793
Epoch: 44
Epoch 1/1
132/132 [=